In [1]:
import pandas as pd
import numpy as np
import re
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import openpyxl 
import os
import re

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
plik = 'dane_nowe.xlsx'

In [4]:
dane = pd.read_excel(plik)

In [5]:
dane.head()

,ID,URL,CALE,PIKSELE,WYSWIETLACZ TYP,CZESTOTLIWOSC ODSWIEZANIA EKRANU,WYSWIETLACZ,APARAT,SYSTEM,PROCESOR,RAM,PAMIEC,CZYTNIK KART,MENU PL,WI-FI,ROZDZIELCZOSC WIDEO,GWARANCJA,WAGA,BLUETOOTH,5G,BATERIA,NAZWA,CENA,WALUTA,OPINIE LICZBA,OCENA
0,1,https://www.euro.com.pl/telefony-komorkowe/sam...,"6,5 """,1600 x 720 pikseli,NaN,60 Hz,16 mln kolorów,48 Mpix + 8 Mpix + 5 Mpix + 2 Mpix,Android 11,MediaTek MT6853 8-rdzeniowy,4 GB,64 GB,tak,tak,tak,4K,24 miesiące,205 g,tak v5.0,tak,5000 mAh,"Smartfon Samsung Galaxy A32 5G - 6,5"" - 48 Mpi...",999.0,PLN,15.0,4.4
1,2,https://www.euro.com.pl/telefony-komorkowe/pan...,"2,4 """,320 x 240 pikseli,NaN,NaN,kolorowy,"0,08 Mpix",producenta,NaN,poniżej 512 MB,NaN,tak,tak,nie,NaN,24 miesiące,106 g,tak,nie,800 mAh,Telefon Panasonic KX-TU400 (czerwony),249.0,PLN,NaN,NaN
2,3,https://www.euro.com.pl/telefony-komorkowe/-sm...,"6,6 """,1600 x 720 pikseli,IPS,60 Hz,HD+,8 Mpix,Android 11 Go,MediaTek A22 4-rdzeniowy,2 GB,32 GB,tak,tak,tak,2K,24 miesiące,198 g,tak,nie,5000 mAh,"Smartfon Infinix SMART 6 HD 2/32GB - 6,6"" - 8 ...",349.0,PLN,NaN,NaN
3,4,https://www.euro.com.pl/telefony-komorkowe/one...,"6,43 """,2408 x 1080 pikseli,AMOLED,90 Hz,Full HD+,50 Mpix + 8 Mpix + 2 Mpix,Android 12,MediaTek Dimensity 1300 8-rdzeniowy,8 GB,128 GB,nie,tak,tak,4K,24 miesiące,190 g,tak v5.2,tak,4500 mAh,"Smartfon OnePlus Nord 2T 5G 8/128GB - 6,43"" - ...",1899.0,PLN,9.0,4.9
4,5,https://www.euro.com.pl/telefony-komorkowe/poc...,"6,67 """,2400 x 1080 pikseli,AMOLED,120 Hz,Full HD+,48 Mpix + 8 Mpix + 2 Mpix,Android 12,Qualcomm Snapdragon 695 8-rdzeniowy,8 GB,256 GB,tak,tak,tak,NaN,24 miesiące,189 g,tak v5.1,tak,5000 mAh,"Smartfon POCO X5 5G 8/256GB - 6,67"" - 48 Mpix ...",1599.0,PLN,5.0,4.9


In [6]:
dane.isnull().sum().sort_values()

ID                                    0
URL                                   0
CALE                                  0
WALUTA                                0
NAZWA                                 0
GWARANCJA                             0
BLUETOOTH                             2
CENA                                  5
5G                                    7
APARAT                               10
PROCESOR                             15
WI-FI                                29
MENU PL                              31
PIKSELE                              43
CZYTNIK KART                         53
SYSTEM                               63
PAMIEC                               77
WAGA                                 97
OCENA                               174
OPINIE LICZBA                       175
RAM                                 191
BATERIA                             192
WYSWIETLACZ                         221
ROZDZIELCZOSC WIDEO                 283
WYSWIETLACZ TYP                     379


In [7]:
dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1391 entries, 0 to 1390
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                1391 non-null   int64  
 1   URL                               1391 non-null   object 
 2   CALE                              1391 non-null   object 
 3   PIKSELE                           1348 non-null   object 
 4   WYSWIETLACZ TYP                   1012 non-null   object 
 5   CZESTOTLIWOSC ODSWIEZANIA EKRANU  895 non-null    object 
 6   WYSWIETLACZ                       1170 non-null   object 
 7   APARAT                            1381 non-null   object 
 8   SYSTEM                            1328 non-null   object 
 9   PROCESOR                          1376 non-null   object 
 10  RAM                               1200 non-null   object 
 11  PAMIEC                            1314 non-null   object 
 12  CZYTNI

piksele ujednolicenie

In [8]:
dane["PIKSELE"] = dane["PIKSELE"].str.replace('pikseli', '') 
dane["PIKSELE"] = dane["PIKSELE"].str.replace('piksele', '') 

piksele ujednolicenie

In [9]:
dane["CALE"] = dane["CALE"].str.replace(' "', '') 

hz ujednolicenie

In [10]:
dane["CZESTOTLIWOSC ODSWIEZANIA EKRANU"] = dane["CZESTOTLIWOSC ODSWIEZANIA EKRANU"].str.replace(' Hz', '') 

wyswietlacz typ ujednolicenie tylko do 0typow IPS, OLED, POLED, AMOLED, INNY

In [11]:
dane['WYSWIETLACZ TYP'] = dane['WYSWIETLACZ TYP'].astype(str)            
dane['WYSWIETLACZ'] = dane['WYSWIETLACZ'].astype(str)

In [12]:
dane.loc[dane['WYSWIETLACZ TYP'] == 'nan', 'WYSWIETLACZ TYP'] = dane['WYSWIETLACZ']

In [13]:
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('AMOLED'), 'WYSWIETLACZ TYP'] = 'AMOLED'

In [14]:
dane['WYSWIETLACZ TYP'] = dane['WYSWIETLACZ TYP'].apply(lambda x: x.upper())

In [15]:
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('POLED'), 'WYSWIETLACZ TYP'] = 'POLED'

In [16]:
dane['WYSWIETLACZ TYP'] = dane['WYSWIETLACZ TYP'].apply(lambda x: x.lower())

In [17]:
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('amoled'), 'WYSWIETLACZ TYP'] = 'AMOLED'
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('poled'), 'WYSWIETLACZ TYP'] = 'POLED'

In [18]:
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('oled'), 'WYSWIETLACZ TYP'] = 'OLED'

In [19]:
dane.loc[dane['WYSWIETLACZ TYP'].str.contains('ips'), 'WYSWIETLACZ TYP'] = 'IPS'

In [20]:
dane.loc[~dane['WYSWIETLACZ TYP'].str.contains('IPS|OLED|POLED|AMOLED'), 'WYSWIETLACZ TYP'] = 'INNY'

In [21]:
unique_values = dane['WYSWIETLACZ TYP'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

value_counts = dane['WYSWIETLACZ TYP'].value_counts(dropna=False)
print(value_counts)

Number of unique values: 5
Unique values:
['AMOLED', 'INNY', 'IPS', 'OLED', 'POLED']
AMOLED    480
INNY      360
IPS       348
OLED      183
POLED      20
Name: WYSWIETLACZ TYP, dtype: int64


OLED typ dla iphone 

In [22]:
dane.loc[(dane['WYSWIETLACZ TYP'] == 'INNY') & (dane['URL'].str.contains('iphone')), 'WYSWIETLACZ TYP'] = 'OLED'

aparat suma pikseli

In [23]:
dane['APARAT'] = dane['APARAT'].astype(str)  

In [24]:
num_regex = re.compile(r'\d+[,.]?\d*')

for i, row in dane.iterrows():
    text = row['APARAT']
    numbers = [float(x.replace(',', '.')) for x in num_regex.findall(text)]
    total = sum(numbers)
    dane.at[i, 'APARAT'] = str(total)

ujednolicenie systemu do Android, IOS, inny

In [25]:
unique_values = dane['SYSTEM'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 31
Unique values:
[' Android 11 ', ' Android 12 ', ' Android 10 ', ' iOS 15 ', ' Android 13 ', ' iOS 16 ', ' producenta ', ' Android 9 Pie ', ' iOS 14 ', ' iOS 13 ', ' EMUI 12 (oparty na systemie Android) HMS ', ' Android 11 Go ', ' Android 8 Oreo ', ' Android 10 + EMUI 10.1 ', ' KaiOS ', ' Android 10 Go ', ' Android 10 + MIUI 11 ', ' Android 9 + EMUI 9 ', ' EMUI 13 (oparty na systemie Android) HMS ', ' Android 13 Go ', ' Android 12 Go ', ' Android 6 Marshmallow ', ' Android 10 + EMUI 10 ', ' Android 8 Oreo Go Edition ', ' Android 10 + One UI 2,0 ', ' Symbian ', ' Android 7 Nougat ', ' Android 9 + EMUI 9.1 ', ' Android 8.1 ', ' iOS 12 ', ' iOS 10 ']


In [26]:
dane['SYSTEM'] = dane['SYSTEM'].astype(str)
dane['SYSTEM'] = dane['SYSTEM'].apply(lambda x: x.upper())

In [27]:
dane.loc[dane['SYSTEM'].str.contains('ANDROID'), 'SYSTEM'] = 'ANDROID'
dane.loc[dane['SYSTEM'].str.contains('IOS'), 'SYSTEM'] = 'IOS'

In [28]:
for i in range(len(dane)):
    if 'ANDROID' not in dane.loc[i, 'SYSTEM'] and 'IOS' not in dane.loc[i, 'SYSTEM']:
        dane.loc[i, 'SYSTEM'] = 'INNY'

5G

In [29]:
dane['5G'] = dane['5G'].astype(str)
dane['5G'] = dane['5G'].str.strip()

In [30]:
dane.loc[dane['5G'].str.contains('nan|nie'), '5G'] = '0'
dane.loc[dane['5G'].str.contains('tak'), '5G'] = '1'
dane['5G'] = dane['5G'].astype(int)

In [31]:
unique_values = dane['5G'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 2
Unique values:
[0, 1]


BLUETOOTH

In [32]:
dane['BLUETOOTH'] = dane['BLUETOOTH'].astype(str)
dane['BLUETOOTH'] = dane['BLUETOOTH'].str.strip()

In [33]:
dane.loc[dane['BLUETOOTH'].str.contains('tak'), 'BLUETOOTH'] = '1'
dane.loc[dane['BLUETOOTH'].str.contains('nan|nie'), 'BLUETOOTH'] = '0'

In [34]:
dane['BLUETOOTH'] = dane['BLUETOOTH'].astype(int)

In [35]:
unique_values = dane['BLUETOOTH'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 2
Unique values:
[1, 0]


GWARANCJA

In [36]:
dane['GWARANCJA'] = dane['GWARANCJA'].str.extract('(\d+)', expand=False)
dane['GWARANCJA'] = dane['GWARANCJA'].astype(int)

In [37]:
unique_values = dane['GWARANCJA'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 3
Unique values:
[24, 12, 36]


WI-FI

In [38]:
dane['WI-FI'] = dane['WI-FI'].astype(str)
dane['WI-FI'] = dane['WI-FI'].str.strip()

In [39]:
dane.loc[dane['WI-FI'].str.contains('nan|nie'), 'WI-FI'] = '0'
dane.loc[dane['WI-FI'].str.contains('tak'), 'WI-FI'] = '1'
dane['WI-FI'] = dane['WI-FI'].astype(int)

In [40]:
unique_values = dane['WI-FI'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 2
Unique values:
[1, 0]


CZYTNIK KART

In [41]:
dane['CZYTNIK KART'] = dane['CZYTNIK KART'].astype(str)
dane['CZYTNIK KART'] = dane['CZYTNIK KART'].str.strip()

In [42]:
dane.loc[dane['CZYTNIK KART'].str.contains('nan|do 1 TB|nie'), 'CZYTNIK KART'] = '0'
dane.loc[dane['CZYTNIK KART'].str.contains('tak'), 'CZYTNIK KART'] = '1'
dane['CZYTNIK KART'] = dane['CZYTNIK KART'].astype(int)

In [43]:
unique_values = dane['CZYTNIK KART'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 2
Unique values:
[1, 0]


MENU PL

In [44]:
dane['MENU PL'] = dane['MENU PL'].astype(str)
dane['MENU PL'] = dane['MENU PL'].str.strip()

In [45]:
dane.loc[dane['MENU PL'].str.contains('nan|nie'), 'MENU PL'] = '0'
dane.loc[dane['MENU PL'].str.contains('tak'), 'MENU PL'] = '1'
dane['MENU PL'] = dane['MENU PL'].astype(int)

In [46]:
unique_values = dane['MENU PL'].value_counts()

print("Number of unique values:", unique_values.count())
print("Unique values:")
print(unique_values.index.tolist())

Number of unique values: 2
Unique values:
[1, 0]


PAMIEC

In [47]:
dane['PAMIEC'] = dane['PAMIEC'].astype(str) 

In [48]:
dane["PAMIEC"] = dane["PAMIEC"].str.replace(' GB', '') 
dane["PAMIEC"] = dane["PAMIEC"].str.replace(',', '.') 

In [49]:
dane['PAMIEC'] = dane['PAMIEC'].str.strip() 

In [50]:
dane['PAMIEC'] = dane['PAMIEC'].astype(float) 

In [51]:
unique_values = dane['PAMIEC'].value_counts()
value_counts = dane['PAMIEC'].value_counts(dropna=False)
print(value_counts)

128.000     548
256.000     277
64.000      243
32.000      114
NaN          77
512.000      62
16.000       23
1000.000     20
4.000         6
0.016         5
0.128         4
0.256         3
8.000         3
0.004         3
0.250         1
0.064         1
0.032         1
Name: PAMIEC, dtype: int64


RAM

In [52]:
unique_values = dane['RAM'].value_counts()
value_counts = dane['RAM'].value_counts(dropna=False)
print(value_counts)

 8 GB               356
 4 GB               298
 6 GB               209
NaN                 191
 poniżej 512 MB      98
 12 GB               82
 3 GB                63
 2 GB                55
 1 GB                17
 16 GB               14
 0,5 GB               8
Name: RAM, dtype: int64


In [53]:
dane['RAM'] = dane['RAM'].astype(str) 

In [54]:
dane["RAM"] = dane["RAM"].str.replace(' GB', '') 
dane["RAM"] = dane["RAM"].str.replace('poniżej 512 MB', '<0,5') 

wartosci RAM dla iphonow

In [55]:
dane.loc[(dane['RAM'] == 'nan') & (dane['URL'].str.contains('iphone')), 'RAM'] = '4'
dane.loc[(dane['RAM'] == '4') & (dane['URL'].str.contains('iphone-12-pro')), 'RAM'] = '6'
dane.loc[(dane['RAM'] == '4') & (dane['URL'].str.contains('iphone-13-pro')), 'RAM'] = '6'
dane.loc[(dane['RAM'] == '4') & (dane['URL'].str.contains('iphone-14')), 'RAM'] = '6'
dane.loc[(dane['RAM'] == '6') & (dane['URL'].str.contains('iphone-14-pro')), 'RAM'] = '8'

In [56]:
dane['RAM'] = dane['RAM'].str.strip() 

In [57]:
unique_values = dane['RAM'].value_counts()
value_counts = dane['RAM'].value_counts(dropna=False)
print(value_counts)

4       397
8       388
6       267
<0,5     98
12       82
3        63
2        55
1        17
16       14
0,5       8
nan       2
Name: RAM, dtype: int64


BATERIA

In [58]:
dane['BATERIA'] = dane['BATERIA'].astype(str) 

In [59]:
dane["BATERIA"] = dane["BATERIA"].str.replace(' mAh', '')

In [60]:
dane.loc[(dane['BATERIA'] == 'nan') & (dane['URL'].str.contains('iphone')), 'BATERIA'] = '2815'
dane.loc[(dane['BATERIA'] == '2815') & (dane['URL'].str.contains('iphone-12-pro')), 'BATERIA'] = '3110'
dane.loc[(dane['BATERIA'] == '2815') & (dane['URL'].str.contains('iphone-13')), 'BATERIA'] = '3095'
dane.loc[(dane['BATERIA'] == '3095') & (dane['URL'].str.contains('iphone-13-pro')), 'BATERIA'] = '4300'
dane.loc[(dane['BATERIA'] == '2815') & (dane['URL'].str.contains('iphone-14')), 'BATERIA'] = '3279'
dane.loc[(dane['BATERIA'] == '3279') & (dane['URL'].str.contains('iphone-14-pro')), 'BATERIA'] = '4322'

In [61]:
dane['BATERIA'] = dane['BATERIA'].str.strip() 

In [62]:
unique_values = dane['BATERIA'].value_counts()
value_counts = dane['BATERIA'].value_counts(dropna=False).sort_values(ascending = False)
print(value_counts)

5000     449
4500     116
4000      74
2815      57
4300      47
3095      42
3279      36
4322      32
6000      32
5020      30
3700      23
4250      22
4400      20
1000      18
4700      18
3300      17
3110      15
800       14
3000      13
4520      13
4800      13
4200      12
970       11
4600       9
1200       9
4020       9
5580       9
3900       8
8580       7
3500       7
5180       7
5160       7
4820       6
8000       6
2000       6
1400       6
900        5
1800       5
2700       5
5260       5
6300       5
4160       5
1450       5
5050       4
4100       4
1500       4
1150       4
4780       4
4310       4
6600       4
3020       3
2500       3
4980       3
5080       3
1715       3
5500       3
nan        3
400        3
0          3
600        3
3520       3
5150       3
1020       3
4470       3
10000      3
5200       2
2990       2
4360       2
4230       2
4085       2
4610       2
2510       2
1810       2
4380       2
4015       2
2200       2
2800       2

WAGA

In [63]:
dane['WAGA'] = dane['WAGA'].astype(str) 

In [64]:
dane["WAGA"] = dane["WAGA"].str.replace(' g', '')
dane["WAGA"] = dane["WAGA"].str.replace(',', '.')

In [65]:
dane['WAGA'] = dane['WAGA'].str.strip() 

In [66]:
dane['WAGA'] = dane['WAGA'].astype('float')

In [67]:
unique_values = dane['WAGA'].value_counts()
value_counts = dane['WAGA'].value_counts(dropna=False).sort_values(ascending = False)
print(value_counts)

NaN      97
190.0    44
195.0    36
199.0    34
172.0    33
203.0    32
194.0    30
205.0    29
193.0    29
196.0    26
240.0    26
174.0    26
189.0    25
202.0    24
206.0    24
204.0    23
228.0    22
185.0    22
192.0    21
141.0    20
187.0    20
155.0    19
162.0    18
175.0    18
167.0    18
180.0    17
198.0    17
209.0    16
183.0    16
179.0    16
133.0    16
148.0    15
182.0    15
188.0    14
181.0    14
191.0    13
201.0    12
158.0    12
177.0    12
169.0    12
186.0    12
226.0    12
178.0    11
144.0    11
165.0    11
178.8     9
184.0     9
197.0     9
173.0     9
237.0     8
170.0     8
220.0     8
203.5     8
176.0     8
215.0     7
325.0     7
110.0     7
210.0     7
254.0     7
208.0     7
159.0     7
271.0     6
234.0     6
218.0     6
168.0     6
161.0     6
171.0     6
235.0     6
73.0      6
150.0     6
297.0     5
227.0     5
239.0     5
179.5     5
81.0      5
85.0      5
214.0     5
157.0     5
90.0      5
273.0     4
380.0     4
270.0     4
207.0     4
163.

zamiana nazw kolumn na male litery bez spacji

In [68]:
dane = dane.rename(columns=lambda x: x.replace(' ', '_').lower())

usuniecie kolumny id

In [69]:
dane = dane.drop(['id'], axis=1)

In [70]:
dane.count()

url                                 1391
cale                                1391
piksele                             1348
wyswietlacz_typ                     1391
czestotliwosc_odswiezania_ekranu     895
wyswietlacz                         1391
aparat                              1391
system                              1391
procesor                            1376
ram                                 1391
pamiec                              1314
czytnik_kart                        1391
menu_pl                             1391
wi-fi                               1391
rozdzielczosc_wideo                 1108
gwarancja                           1391
waga                                1294
bluetooth                           1391
5g                                  1391
bateria                             1391
nazwa                               1391
cena                                1386
waluta                              1391
opinie_liczba                       1216
ocena           

usuniecie kolumny 'wyswietlacz' - zbyt roznorodne dane

In [71]:
dane = dane = dane.drop(['wyswietlacz'], axis=1)

dane bez opinii

In [72]:
dane_bez_opinii = dane.drop(['opinie_liczba', 'ocena'], axis=1)

usuniecie wierszów gdzie liczba nulli jest wieksza niz 2 (nie uwzgledniajac kolumn liczba_opinii, ocena)

In [73]:
null_counts = dane_bez_opinii.isnull().sum(axis=1)

In [82]:
rows_to_drop = null_counts[null_counts > 2].index
#print(rows_to_drop)
print(len(rows_to_drop))

102


In [75]:
dane = dane.drop(rows_to_drop)

INDEX

In [77]:
dane = dane.reset_index()

In [78]:
dane['id'] = dane.index + 1

In [79]:
dane = dane.drop(['index'], axis=1)

In [81]:
dane.to_excel('dane_final.xlsx', index=False)